Аббревиатура XML расшифровывается как eXtensible Markup Language — расширяемый язык разметки. Он (язык) позволяет описывать документы, используя теги.
В примере выше довольно жёсткая структура: у нас есть меню  — тег <menu>, где хранятся объекты, то есть конкретные блюда, помеченные тегом <dish>. У каждого из блюд есть параметр name, в котором прописано имя блюда. Можно считать, что <dish> — класс этого объекта.
Внутри каждого объекта-блюда находится набор значений, которые тоже задаются тегами. Например, внутри тега <price> находится значение 20, означающее цену данного блюда. Тег <price> и другие теги внутри можно рассматривать как атрибуты класса dish.
Данные в формате XML имеют древовидную структуру. 

Что такое дерево? Это структура, которая имеет узлы и связи между ними. Самый верхнеуровневый узел называется корнем, а всё, что находится в самом низу, называется листьями. 

В примере из прошлого юнита корнем является <menu>, а листьями, например, <price> и <weight>.

Кроме того, у <menu> есть дети (потомки) — это два узла <dish>, имеющие разное значение атрибута <name>.

Таким образом, в файле используется набор тегов, внутри которых могут находиться другие теги со своими значениями.


ИЗВЛЕКАЕМ КОНТЕНТ ИЗ XML-ФАЙЛА

Для работы с XML-файлами мы будем использовать модуль ElementTree , входящий в стандартный пакет xml. Этот модуль позволит нам «перемещаться» по дереву XML и смотреть, что находится в каждом его узле, начиная с корня и заканчивая листьями.

In [3]:
import pandas as pd
# Импортируем модуль ElementTree
import xml.etree.ElementTree as ET

tree = ET.parse('data/menu.xml')


КОРЕНЬ
Запишем в переменную root корневой узел дерева tree и посмотрим, как выглядит содержимое переменной root, для чего выполним код:


In [4]:
root = tree.getroot()
display(root)

<Element 'menu' at 0x00000287332CCA40>

Какой тип у этого объекта? Если мы вызовем встроенный в Python метод type() и передадим ему root , то увидим, что это тип xml.etree.ElementTree.Element. Такой тип будет у любого узла в дереве.

In [5]:
display(type(root))


xml.etree.ElementTree.Element

ПОТОМКИ

Для того чтобы посмотреть список потомков корневого узла, выполним следующий код:

In [6]:
display(list(root))

[<Element 'dish' at 0x00000287332CCB80>,
 <Element 'dish' at 0x00000287332CCC20>]

Для того чтобы получить список потомков второго блюда в нашем меню и вывести его на экран, выполним код:

In [7]:
display(list(root[1]))

[<Element 'price' at 0x00000287332CCD10>,
 <Element 'weight' at 0x00000287332CCD60>,
 <Element 'class' at 0x00000287332CCDB0>]

АТРИБУТЫ И ТЕГИ

Мы можем непосредственно обратиться к атрибутам, используя attrib.
Выведем на экран атрибуты первого блюда из меню:


In [8]:
display(root[0].attrib)

{'name': 'Кура'}

В XML-узлах часто хранятся количественные показатели. Эти показатели хранятся в виде текста, и прочитать их можно, обратившись к атрибуту text у соответствующего объекта типа ElementTree.Element.

Например, возьмём узел price первого блюда из меню:

In [9]:
display(root[0][0])

<Element 'price' at 0x00000287332CCBD0>

Теперь прочитаем значение этого узла с помощью text:

In [10]:
display(root[0][0].text)

'40'

Все значения в XML, даже числовые, хранятся как строки, поэтому преобразовывать их к нужному типу вам нужно самим.

Если вы хотите прочитать наименование тега конкретного узла, необходимо использовать tag. Например, получим наименование тега корневого узла:

In [11]:
display(root[0][2].tag)

'class'

ИСПОЛЬЗОВАНИЕ ЦИКЛОВ

Итак, мы научились обращаться к отдельным узлам дерева, представляющего XML-структуру, и извлекать информацию о его атрибутах, значении и потомках.

In [12]:
for dish in root:
    for param in dish:
        print(dish.attrib['name'], param.tag, param.text)
    print()

Кура price 40
Кура weight 300
Кура class Мясо

Греча price 20
Греча weight 200
Греча class Крупа



ЗАГРУЖАЕМ ДАННЫЕ ИЗ XML-ФАЙЛА В DATAFRAME

Реализуем следующий алгоритм:

Загрузить XML-структуру файла menu.xml в переменную root.
Создать пустой список df_list (в него будем добавлять строчки итоговой таблицы).
Заранее создать список column_names с именами столбцов — название блюда (name), его цена (price), вес (weight) и класс (class).
В цикле организовать обход xml-дерева из корня по всем потомкам.
На каждой итерации цикла сформировать в виде списка строку таблицы, содержащую информацию: наименование блюда (атрибут name узла dish) и значения потомков этого узла — узлов price, weight, class.
Добавить сформированную строку в список df_list, используя метод append().
Сформировать из вложенного списка DataFrame. Имена для столбцов взять из списка column_names.

In [14]:
import xml.etree.ElementTree as ET
tree = ET.parse('data/menu.xml')
root = tree.getroot()

import pandas as pd
column_names = ['name', 'price', 'weight', 'class']
df_list = []

for dish in root:
    row = [dish.attrib['name'], dish[0].text, dish[1].text, dish[2].text]
    df_list.append(row)
    df = pd.DataFrame(df_list, columns=column_names)
display(df)

,name,price,weight,class
0,Кура,40,300,Мясо
1,Греча,20,200,Крупа


СОЗДАЁМ XML-ФАЙЛ

Чтобы создать корень дерева, используем метод Element() из класса ElementTree:

In [15]:
import xml.etree.ElementTree as ET

new_root = ET.Element('menu')
display(new_root)

<Element 'menu' at 0x00000287332D1530>

Теперь мы можем добавлять новые узлы в наше дерево, используя метод SubElement() из того же класса.

В метод SubElement() мы передали первым аргументом узел, к которому добавляем потомка, вторым аргументом — наименование нового тега (dish),  третьим аргументом — наименование атрибута нового узла( name ) и его значение.

In [16]:
dish1 = ET.SubElement(new_root, 'dish', name='Кура')

dish2 = ET.SubElement(new_root, 'dish', name='Греча')

display(list(new_root))


[<Element 'dish' at 0x0000028733317510>,
 <Element 'dish' at 0x00000287333174C0>]

Добавим в создаваемую структуру по три потомка (атрибута) к двум новым узлам, которые будут содержать информацию о блюде — о его цене (price), весе (weight) и классе (class), а также значение этих атрибутов:

In [17]:
price1 = ET.SubElement(dish1, "price").text = "40"
weight1 = ET.SubElement(dish1, "weight").text = "300"
class1 = ET.SubElement(dish1, "class").text = "Мясо"
display(list(dish1))

price2 = ET.SubElement(dish2, "price").text = "20"
weight2 = ET.SubElement(dish2, "weight").text = "200"
class2 = ET.SubElement(dish2, "class").text = "Крупа"
display(list(dish2))

[<Element 'price' at 0x0000028733326110>,
 <Element 'weight' at 0x00000287333261B0>,
 <Element 'class' at 0x0000028733326200>]

[<Element 'price' at 0x0000028721EB93A0>,
 <Element 'weight' at 0x0000028733317150>,
 <Element 'class' at 0x0000028723BEE110>]

Созданная нами структура полностью идентична структуре исходного XML-файла.

In [18]:
for dish in new_root:    
    for param in dish:
        print(dish.attrib['name'], param.tag, param.text)
    print()

Кура price 40
Кура weight 300
Кура class Мясо

Греча price 20
Греча weight 200
Греча class Крупа



СОХРАНЕНИЕ XML-ФАЙЛА

Преобразуем созданный нами объект типа ElementTree.Element в строку c помощью метода tostring(), передав наше новое дерево как аргумент. Сохраним эту строку на диске, используя стандартные средства Python::

In [22]:
new_root_string = ET.tostring(new_root)

with open('data/new_menu.xml', 'wb') as f:
    f.write(new_root_string)

Возможно, вы увидите проблему, связанную с кодировкой. Что делать в этом случае? Как вариант — записать файл, используя сам класс ElementTree() :

Для этого мы передаём в класс ElementTree() наше дерево (не его строковое представление) и вызываем метод write(). В метод мы передаём путь к новому файлу и нужную нам кодировку.

In [ ]:
ET.ElementTree(new_root).write('new_menu_good.xml', encoding="utf-8")